# Electric Vehicle Charge Scheduling MDP

### Dependencies

You will need to install POMDPs and POMDPToolbox locally before being able to run this notebook. This can be done by running the following in your local Julia 1.0 REPL: 
    - Pkg.add("POMDPs")
    - Pkg.add("POMDPToolbox")

In [ ]:
using POMDPs, POMDPToolbox

### State Structure
Define state structure / make initial constructor

In [ ]:
mutable struct evState
    p::Vector{Bool} # array of whether cars are present
    c::Vector{Int64} # array of charge in each car
    renew::Int64 # renewable energy level
    t::Int64 # time
    done::Bool # are we in a terminal state
end

# initial state constructor
evState(p,c,renew::Int64, renew::Int64, t::Int64) = evState(p,c,renew,t,false)

### MDP Structure
Define MDP structure with everything you would need / make initial constructor

In [ ]:
struct evMDP <: MDP{evState,Vector{Bool}} 
    n::Int64 # number of cars
    T::Int64 # number of timesteps
    renew_levels::Int64 # number of renewable mixture levels, 0:renew_levels
    charge_levels::Int64 # number of charge levels, 0:charge_levels
    λ::Float64 # terminal reward weighting
end

# we use key worded arguments so we can change any of the values we pass in 
function evMDP(;n::Int64 = 3 # number of cars
                T::Int64 = 6 # number of timesteps
                renew_levels::Int64 = 4, # number of renewable mixture levels, 0:renew_levels
                charge_levels::Int64 = 4, # number of charge levels, 0:charge_levels
                λ::Float64 = 100) #terminal reward weighting
    return evMDP(n, T, renew_levels, charge_levels, λ)
end


### States
Define all possible states

In [ ]:
# convert number to an array of numbers using requested base system, array length
function num2array(number,base,array_length)
    base==2 ? finalarray = zeros(Bool, array_length) : finalarray = zeros(Int64, array_length)
    idx=1
    while number > 0
        finalarray[idx] = rem(number,base)
        number = div(number,base)
        idx+=1
    end
    return finalarray
end

function POMDPs.states(mdp::evMDP)
    s = GridWorldState[] # initialize an array of GridWorldStates
    
    # add every possible state. This includes every possible combination of present/charge array
    
    for iP = 0:(2^mdp.n-1)
        present = num2array(iP,2,mdp.n)
        
        for iC = 0:((mdp.charge_levels+1)^mdp.n-1)
            charge = num2array(iC,(mdp.charge_levels+1),mdp.n)
            
            for rl=0:mdp.renew_levels, t=1:mdp.T
            
                # if in final time, make sure the done flag is set on
                t==mdp.T ? push!(s,evState(present, charge, rl, t, true)) : push!(s,evState(present, charge, rl, t)) 
            end
        end
    end
    return s
end

### Actions
Define all possible action vectors

In [ ]:
function POMDPs.actions(mdp::evMDP)
    # initialize empty action space a
    a = []    
    # populate with all combinations of actions, ex [true, false, true, true]
    for iA=0:(2^mdp.n-1)
        push!(a,num2array(iA,2,mdp.n))
    end
    return a
end
    

### Reward Function
Define the reward function

In [ ]:
function POMDPs.reward(mdp::evMDP, state::evState, action::Vector{Bool}, statep::evState)
    return r
end

### Transition Function
Define the next-state transition probabilities (this is the hard one)

In [ ]:
function POMDPs.transition(mdp::evMDP, state::evState, action::Vector{Bool})
end

### Miscellaneous Functions
Define other functions that POMDPs.jl needs

In [ ]:
POMDPs.n_states(mdp::evMDP) = 2^mdp.n*(mdp.charge_levels+1)^mdp.n*(mdp.renew_levels+1)*mdp.T
POMDPs.n_actions(mdp::evMDP) = 2^mdp.n
POMDPs.discount(mdp::evMDP) = 1
POMDPs.isterminal(mdp::evMDP, s::evState) = s.done

In [ ]:
# define state and action indexing
function POMDPs.state_index(mdp::evMDP, state::evState)
end
function POMDPs.action_index(mdp::evMDP, act::Vector{Bool})
end

### Implement Solvers / Simulators